In [3]:
from pyspark.sql import SparkSession


# to start
First we have to create a spark sessionand we have to give it an appname and a master
the master will tell spark where the spark job needs to run, here it will be local because we are running it on our local machine.
Typically you would put an url for a spark cluster/mesos cluster or YARN if you were using that.


In [4]:
spark = SparkSession.builder.master("local").appName("demo application").getOrCreate()

create a dataframe to interact with (structured data in our case)
we read it from a csv but if it is an array you can use the parelilize method to create a dataframe from an array.
we need to also pass a schema when creating a dataframe (column names) our csv already has the headers so it will do it automatically also datatypes will be asssigned automatically because we give the option inferschema

the \ are used so that cwe can make the code more readable since laot of .methods are being chained behind eachother.

we have to specify the path and then load the data into a dataframe.


In [5]:
data = spark.read.format('csv').option('inferSchema', 'True').\
    option('header', 'True').\
        option('path', './data/operations_management.csv').\
            load()

In [7]:
data.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: string (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: string (nullable = true)



as you can see it has created the dataframe with the correct datatypes for every column.


# TRANSFORMING DATA
like in pandas, you do not change the original data in the dataframe.

here we will select two columns, industry and value because in this example we are interested in that.

we filter so that we only have the industries that are worth over 1000 and then we sort them in descending order to get the most valuable ones first in this dataframe.

In [8]:
# create ac opy and apply transformation logic  to the dataframe
data2 = data.select("industry", "value").\
    filter(data["value"] > 10000).\
    orderBy(data["value"].desc())






In [9]:
data2.show()

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
|   total|31134|
|   total|30624|
|   total|30543|
|   total|30396|
|   total|30099|
|   total|29994|
|   total|29775|
|   total|28473|
|   total|27846|
|   total|26010|
|   total|25977|
|   total|25887|
|   total|25500|
|   total|25434|
|   total|25221|
+--------+-----+
only showing top 20 rows



the industry column is showing "total" for alot of rows which means the dataset had columns mixed with total values of certain industries mixed with the actual industry names, we need to filter this out.

In [10]:
data2 = data.select("industry", "value").\
    filter((data["value"] > 2000) & (data["industry"] != "total")).\
    filter((data["value"] > 2000) & (data["industry"] != "total")).\
    orderBy(data["value"].desc())


In [11]:
data2.show(truncate=False)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
|        Construction| 4959|
|Accommodation & f...| 4950|
|        Construction| 4686|
|        Construction| 4668|
|        Construction| 4665|
|       Manufacturing| 4662|
|       Manufacturing| 4632|
|        Construction| 4575|
|        Construction| 4566|
|Professional, sci...| 4476|
|Professional, sci...| 4470|
|        Retail trade| 4434|
|        Retail trade| 4434|
|Accommodation & f...| 4251|
|Accommodation & f...| 4176|
+--------------------+-----+
only showing top 20 rows



There is alot of duplicate industries, these are probably tied to different company names but we want the value for the industries themselves, so we need to group by the industry name.

to do that we will need to import aggregation functions from the pyspark library

In [24]:
from pyspark.sql.functions import col, sum as _sum


in SQL you would put the aggregation function in the select statement however in pyspark you have a seperate clause for it after the groupby.

In [25]:
data2 = data.select("industry", "value").\
    filter((data["value"] > 2000) & (data["industry"] != "total")).\
    groupBy("industry").\
    agg(_sum("value").alias("total_industry_value")).\
    orderBy("total_industry_value", ascending=False)

In [26]:
data2.show(truncate=False)

+----------------------------------------------+--------------------+
|industry                                      |total_industry_value|
+----------------------------------------------+--------------------+
|Construction                                  |177639              |
|Accommodation & food services                 |150768              |
|Professional, scientific, & technical services|114789              |
|Manufacturing                                 |100275              |
|Retail trade                                  |76461               |
|Other professional scientific                 |58602               |
|Agriculture, forestry, & fishing              |38547               |
|Health care & social assistance               |17601               |
|Wholesale trade                               |16431               |
|Agriculture                                   |4149                |
+----------------------------------------------+--------------------+



# SQL in apache spark.

SQL queries will not run directly on apache spark dataframes themselves, you will need to create a view or temporary view from the dataframe in order to do SQL on it.

In [35]:
data.createOrReplaceTempView("data2_view")

when creating a temporary view, it will ge tregistered in the spark SQL catalog and any SQL queries using the spark.sql() method will reference this view you created by the name you gave that view and will show up as a temporary table in the spark catalog if you call listtables().

In [40]:
print(spark.catalog.listTables())   

[Table(name='data', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='data2_view', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='data3', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True), Table(name='data4', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


In [49]:
spark.sql("""
          SELECT industry, value 
          FROM data2_view 
          WHERE industry != 'total' AND value > 2000
          ORDER BY value DESC
          """).show(truncate=False) 

+----------------------------------------------+-----+
|industry                                      |value|
+----------------------------------------------+-----+
|Construction                                  |6030 |
|Construction                                  |5904 |
|Construction                                  |5229 |
|Accommodation & food services                 |5058 |
|Construction                                  |4965 |
|Construction                                  |4959 |
|Accommodation & food services                 |4950 |
|Construction                                  |4686 |
|Construction                                  |4668 |
|Construction                                  |4665 |
|Manufacturing                                 |4662 |
|Manufacturing                                 |4632 |
|Construction                                  |4575 |
|Construction                                  |4566 |
|Professional, scientific, & technical services|4476 |
|Professio

In [ ]:
data.createOrReplaceTempView("data2_view")

In [48]:
spark.sql("""
          SELECT industry, SUM(value) AS total_industry_value 
          FROM data2_view 
          WHERE industry != 'total' AND value > 2000
          GROUP BY industry
          ORDER BY total_industry_value DESC
          """).show(truncate=False) 



+----------------------------------------------+--------------------+
|industry                                      |total_industry_value|
+----------------------------------------------+--------------------+
|Construction                                  |177639              |
|Accommodation & food services                 |150768              |
|Professional, scientific, & technical services|114789              |
|Manufacturing                                 |100275              |
|Retail trade                                  |76461               |
|Other professional scientific                 |58602               |
|Agriculture, forestry, & fishing              |38547               |
|Health care & social assistance               |17601               |
|Wholesale trade                               |16431               |
|Agriculture                                   |4149                |
+----------------------------------------------+--------------------+



Now that is the basics of what you can do with dataprocessing using apache spark.